<a href="https://colab.research.google.com/github/cybr-dev/cybr-dev/blob/main/DonerDaily.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.2 MB/s

In [16]:
import pandas as pd
import numpy as np
data = pd.read_csv('update.csv')
data['DATE'] = pd.to_datetime(data['DATE'])
data.head()
# extract month and year from dates**
data['Month'] = [i.month for i in data['DATE']]
data['Year'] = [i.year for i in data['DATE']]
data['Day'] = [i.day for i in data['DATE']]

# create a sequence of numbers
data['Series'] = np.arange(1,len(data)+1)

# drop unnecessary columns and re-arrange
data = data[['Series', 'Year', 'Month', 'Day', 'WEEKDAY', 'NUM_ORDERS']]

# check the head of the dataset**
data.head()

# import the regression module**
from pycaret.regression import *

# initialize setup**
s = setup(data = data, target = 'NUM_ORDERS', fold_strategy = 'timeseries', numeric_features = ['Year', 'Series'], fold = 3, transform_target = True, session_id = 123, data_split_shuffle = False)

,Description,Value
0,Session id,123
1,Target,NUM_ORDERS
2,Target type,Regression
3,Original data shape,"(1655, 6)"
4,Transformed data shape,"(1655, 12)"
5,Transformed train set shape,"(1158, 12)"
6,Transformed test set shape,"(497, 12)"
7,Numeric features,2
8,Categorical features,1
9,Preprocess,True


In [17]:
best = compare_models()
print(best)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,171.8394,65817.6884,254.7954,0.5707,1.6143,18.3301,0.2833
huber,Huber Regressor,232.9124,99634.9692,296.6299,0.3431,1.7868,27.2078,0.1300
dummy,Dummy Regressor,272.7046,162297.5683,402.8375,-0.0468,2.1459,47.9021,0.0933
omp,Orthogonal Matching Pursuit,412.7719,254786.9035,489.1771,-0.6789,2.2098,39.9707,0.1000
rf,Random Forest Regressor,383.7460,312241.1595,463.0132,-1.1081,2.7980,13.9040,0.6567
dt,Decision Tree Regressor,422.3806,342085.1073,515.0336,-1.2968,2.9828,14.0261,0.1367
ada,AdaBoost Regressor,433.6579,354570.9407,531.5045,-1.3780,2.8563,25.0658,0.1800
knn,K Neighbors Regressor,483.0478,442812.3069,637.4877,-1.9259,3.1678,36.2558,0.1200
par,Passive Aggressive Regressor,600.0726,607430.1924,702.3413,-2.9163,2.1091,38.4098,0.0900
xgboost,Extreme Gradient Boosting,550.8137,1577014.8701,886.2899,-9.7336,2.3335,24.2965,0.1600


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

ExtraTreesRegressor(n_jobs=-1, random_state=123)


In [10]:
prediction_holdout = predict_model(best);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,145.3131,49913.0185,223.4122,0.7185,1.7174,25.1357


In [44]:
import plotly.express as px
# generate predictions on the original dataset**
predictions = predict_model(best, data=data)



# add a date column in the dataset**
predictions['Date'] = pd.to_datetime(predictions[['Year', 'Month', 'Day']])
predictions.sort_values(by=['Date'], inplace=True)
predictions.head()

data['Date'] = pd.to_datetime(predictions[['Year', 'Month', 'Day']])
data.sort_values(by=['Date'], inplace=True)

# line plot**
fig = px.line(predictions.iloc[-100:], x='Date', y=["NUM_ORDERS"], template = 'plotly_dark')
fig.add_scatter(x=data['Date'].iloc[-100:], y=data['NUM_ORDERS'], mode='lines')



print(len(predictions['Date']))
print(len(predictions['NUM_ORDERS']))


fig.show()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,39.0597,13229.1336,115.0180,0.9155,0.9320,7.5056


1655
1655


In [34]:
final_best = finalize_model(best)
future_dates = pd.date_range(start = '2009-01-01', end = '2024-03-29', freq = 'MS')

future_df = pd.DataFrame()

future_df['Month'] = [i.month for i in future_dates]
future_df['Year'] = [i.year for i in future_dates]
future_df['Series'] = np.arange(145,(145+len(future_dates)))

future_df.head()
predictions_future = predict_model(final_best, data=future_df)
predictions_future.head()

concat_df = pd.concat([data,predictions_future], axis=0)
concat_df_i = pd.date_range(start='1949-01-01', end = '1965-01-01', freq = 'MS')
concat_df.set_index(concat_df_i, inplace=True)

fig = px.line(concat_df, x=concat_df.index, y=["NUM_ORDERS", "Label"], template = 'plotly_dark')
fig.show()

KeyError: "['Day', 'WEEKDAY'] not in index"